In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

df_eddies = pd.read_pickle('/srv/scratch/z5297792/Clim_data/df_eddies_processed.pkl')
with open('/srv/scratch/z5297792/Clim_data/Sample_Data/dic_sample_vert_info.pkl', 'rb') as f:
    dic_sample = pickle.load(f)
sample_eddies = [8, 896, 2504, 2749, 382, 926, 1394, 1967]

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import plot_ellipse


In [2]:
import netCDF4 as nc
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time

# Field Data

fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]
z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))[150, 150, :]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')

res = 1  # 1 km resolution
x_new = np.arange(0, x_grid[-1], res)
y_new = np.arange(0, y_grid[-1], res)
X_new, Y_new = np.meshgrid(x_new, y_new, indexing='ij')
new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

interp_lon = RegularGridInterpolator((x_grid, y_grid), lon_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)
interp_lat = RegularGridInterpolator((x_grid, y_grid), lat_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)

lon_new = interp_lon(new_points).reshape(len(x_new), len(y_new))
lat_new = interp_lat(new_points).reshape(len(x_new), len(y_new))


#### Side Profiles

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(10,5), sharey=True)
df_means = pd.DataFrame([
    {
        'Eddy': eddy,
        'x_mean': df_all['x'].mean(),
        'y_mean': df_all['y'].mean()
    }
    for eddy, days in dic_sample.items()
    for df_all in [pd.concat(
        [df0 for df0 in days.values() 
         if not df0.empty and not df0.isna().all().all()],
        ignore_index=True
    )]
])

for e, eddy in enumerate(dic_sample.keys()):
    ax = axs[e // 4, e % 4]
    dic = dic_sample[eddy]
    pmean = df_means[df_means['Eddy']==eddy].iloc[0]
    for d, day in enumerate(dic.keys()):
        df = dic[day].copy()
        if ('1967' in eddy) or ('382' in eddy):
            ax.plot(df['x'], -df['Depth']/1000)
            ax.set_xlabel('x (km)')
        else:
            ax.plot(df['y'], -df['Depth']/1000)
            ax.set_xlabel('y (km)')
    xlim = ax.get_xlim()
    
    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean']-y_grid)==np.min(np.abs(pmean['y_mean']-y_grid)))[0][0]
        ax.plot(x_grid, h[:, jc]/1000, 'k')
    else:
        ic = np.where(np.abs(pmean['x_mean']-x_grid)==np.min(np.abs(pmean['x_mean']-x_grid)))[0][0]
        ax.plot(y_grid, h[ic, :]/1000, 'k')

    ylim = ax.get_ylim()

    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean'] - y_grid) == np.min(np.abs(pmean['y_mean'] - y_grid)))[0][0]
        ax.fill_between(x_grid, h[:, jc] / 1000, 10, color='k')
    else:
        ic = np.where(np.abs(pmean['x_mean'] - x_grid) == np.min(np.abs(pmean['x_mean'] - x_grid)))[0][0]
        ax.fill_between(y_grid, h[ic, :] / 1000, 10, color='k')

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
            
    cyc = df_eddies[df_eddies['ID']==int(eddy[4:])].iloc[0]['Cyc']
    ax.set_title(f'{cyc}{eddy[4:]}')
    if e % 4 == 0:
        ax.set_ylabel('Depth (km)')
axs[0,0].invert_yaxis()

# for ax in axs.flat:
#     xmin, xmax = ax.get_xlim()
#     ax.set_xticks(np.arange(np.floor(xmin / 100) * 100, np.ceil(xmax / 100) * 100 + 1, 100))

plt.tight_layout()


#### Depths

In [ ]:
for e, eddy in enumerate(dic_sample.keys()):
    dic = dic_sample[eddy]
    depths = []
    for d, day in enumerate(dic.keys()):
        df = dic[day].copy()
        if len(df):
            depths.append(-df.iloc[-1]['Depth']/1000)
        else:
            depths.append(0)
    plt.plot(depths)


#### Ellipticity

In [ ]:
def smooth(x, y, num=1000, window=100):
    from scipy.interpolate import interp1d
    from scipy.ndimage import uniform_filter1d
    x = np.asarray(x); y = np.asarray(y)
    # Step 1: interpolate to uniform y
    y_uniform = np.linspace(y.min(), y.max(), num)
    f_interp = interp1d(y, x, kind='linear', fill_value='extrapolate')
    x_uniform = f_interp(y_uniform)
    # Step 2: smooth x on the uniform y grid
    x_smooth_uniform = uniform_filter1d(x_uniform, size=window)
    # Step 3: interpolate back to original y
    f_smooth = interp1d(y_uniform, x_smooth_uniform, kind='linear', fill_value='extrapolate')
    x_smooth = f_smooth(y)

    return x_smooth
    

##### Surface

In [ ]:
from matplotlib.cm import cool
from matplotlib.colors import Normalize

fig, axs = plt.subplots(2, 4, figsize=(13, 8))

for e, eddy in enumerate(dic_sample):
    ax = axs[e // 4, e % 4]
    xs, ys, Qs, Rcs, ds = [], [], [], [], []
    for d, day in enumerate(dic_sample[eddy]):
        df = dic_sample[eddy][day]
        if df.empty:
            continue
        xs.append(df.iloc[0]['x'])
        ys.append(df.iloc[0]['y'])
        Qs.append(df.iloc[0]['Q'])
        Rcs.append(df.iloc[0]['Rc'])
        ds.append(d)

    if not ds:
        ax.set_title(eddy)
        continue

    if ('382' in eddy) or ('1967' in eddy):
        ax.plot(ys, xs, linewidth=.5, color='lightgray', alpha=.7)
        ax.set_xlabel('y (km)')
        ax.set_ylabel('x (km)')
    else:
        ax.plot(xs, ys, linewidth=.5, color='lightgray', alpha=.7)
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')

    days = np.array(ds)
    window = 5
    sq11 = smooth([q[0,0] for q in Qs], days, num=len(days), window=window)
    sq12 = smooth([q[0,1] for q in Qs], days, num=len(days), window=window)
    sq22 = smooth([q[1,1] for q in Qs], days, num=len(days), window=window)
    sRc  = smooth(Rcs,       days, num=len(days), window=window)

    norm = Normalize(vmin=days.min(), vmax=days.max())
    cmap = cool
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array(days)

    for xi, yi, a, b, c, scale, day in zip(xs, ys, sq11, sq12, sq22, sRc, ds):
        Qm = np.array([[a, b], [b, c]])
        xe, ye = plot_ellipse(Qm, (xi, yi), scale=scale)
        if ('382' in eddy) or ('1967' in eddy):
            ax.plot(ye, xe, color=cmap(norm(day)), alpha=.8)
        else:
            ax.plot(xe, ye, color=cmap(norm(day)), alpha=.8)

    cbar = fig.colorbar(
        sm, ax=ax,
        orientation='vertical',
        fraction=0.046,
        pad=0.04
    )
    cbar.set_label('Day')
    cbar.ax.yaxis.set_ticks_position('right')
    cbar.ax.yaxis.set_label_position('right')

    ax.axis('equal')
    cyc = df_eddies.loc[df_eddies['ID'] == int(eddy[4:]), 'Cyc'].iloc[0]
    ax.set_title(f'{cyc}{eddy[4:]}')


plt.tight_layout()
plt.show()


##### Depth

In [ ]:
sample_eddies

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import cool, viridis
from matplotlib.colors import Normalize

# pick your eddy and day-index
eddy = 382
day_dx = 10

eddy = f'Eddy{eddy}'

day_key = list(dic_sample[eddy].keys())[day_idx]
df = dic_sample[eddy][day_key]

# raw data
xs = df['x'].values
ys = df['y'].values
Qs = df['Q'].values
Rcs = df['Rc'].values
psi0s = df['psi0'].values
depths = -df['Depth'].values / 1000  # in km

# smooth against frame index t
window = 3
sq11  = smooth([q[0,0] for q in Qs],   depths, num=len(depths), window=window)
sq12  = smooth([q[0,1] for q in Qs],   depths, num=len(depths), window=window)
sq22  = smooth([q[1,1] for q in Qs],   depths, num=len(depths), window=window)
sRc   = smooth(Rcs,                    depths, num=len(depths), window=window)
spsi0 = smooth(psi0s,                  depths, num=len(depths), window=window)

# prepare figure
fig = plt.figure(figsize=(8,6))
ax  = fig.add_subplot(111, projection='3d')

# depth colormap
norm = Normalize(vmin=depths.min(), vmax=depths.max())
cmap = viridis
sm   = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array(depths)

# plot each smoothed ellipse at its depth
ax.plot(xs, ys, depths)
for x0, y0, q11, q12, q22, Rc, z0 in zip(xs, ys, sq11, sq12, sq22, sRc, depths):
    Qm = np.array([[q11, q12],[q12, q22]])
    xe, ye = plot_ellipse(Qm, (x0, y0), scale=Rc)
    ze = np.full_like(xe, z0)
    ax.plot(xe, ye, ze, color=cmap(norm(z0)), alpha=0.8)

# colourbar and labels
cbar = fig.colorbar(sm, ax=ax, pad=0.1)
cbar.set_label('Depth (km)')
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Depth (km)')
ax.set_title(f'{eddy}')
ax.invert_zaxis()
ax.view_init(elev=5, azim=120)

plt.tight_layout()
plt.show()


In [ ]:
df_means = pd.DataFrame([
    {
        'Eddy': eddy,
        'x_mean': df_all['x'].mean(),
        'y_mean': df_all['y'].mean()
    }
    for eddy, days in dic_sample.items()
    for df_all in [pd.concat(
        [df0 for df0 in days.values() 
         if not df0.empty and not df0.isna().all().all()],
        ignore_index=True
    )]
])


In [ ]:
def side_profile_plotter(eddy, days):
    eddy = f'Eddy{eddy}'
    
    pmean = df_means[df_means['Eddy']==eddy].iloc[0]
    
    fig, ax = plt.subplots(1, 1, figsize=(12,6))
    
    dic = dic_sample[eddy]

    for day in dic.keys():
        if int(day[3:]) not in days:
            df = dic[day].copy()
            if ('1967' in eddy) or ('382' in eddy):
                ax.plot(df.x, -df.Depth/1000, linewidth=.5, color='k', alpha=.2)
            else:
                ax.plot(df.y, -df.Depth/1000, linewidth=.5, color='k', alpha=.2)
    
    colors = plt.get_cmap('tab10').colors
    for d, day_idx in enumerate(days):
        
        day_key = list(dic_sample[eddy].keys())[day_idx]
        df = dic[day_key].copy()
    
        if len(df):
            
            # raw data
            xs = df['x'].values
            ys = df['y'].values
            Qs = df['Q'].values
            Rcs = df['Rc'].values
            psi0s = df['psi0'].values
            depths = -df['Depth'].values / 1000  # in km
            
            # smooth against frame index t
            window = 50
            sq11  = smooth([q[0,0] for q in Qs],   depths, window=window)
            sq12  = smooth([q[0,1] for q in Qs],   depths, window=window)
            sq22  = smooth([q[1,1] for q in Qs],   depths, window=window)
            sRc   = smooth(Rcs,                    depths, window=window)
            spsi0 = smooth(psi0s,                  depths, window=window)
            
            # plot each smoothed ellipse at its depth
            if ('1967' in eddy) or ('382' in eddy):
                ax.plot(xs, depths, color=colors[d % 10], label=day_idx)
            else:
                ax.plot(ys, depths, color=colors[d % 10], label=day_idx)
            for x0, y0, q11, q12, q22, Rc, z0 in zip(xs, ys, sq11, sq12, sq22, sRc, depths):
                Qm = np.array([[q11, q12],[q12, q22]])
                xe, ye = plot_ellipse(Qm, (x0, y0), scale=Rc)
                ze = np.full_like(xe, z0)
                if ('1967' in eddy) or ('382' in eddy):
                    ax.plot(xe, ze, color=colors[d % 10], alpha=0.8)
                else:
                    ax.plot(ye, ze, color=colors[d % 10], alpha=0.8)
                
    xlim = ax.get_xlim()
    
    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean']-y_grid)==np.min(np.abs(pmean['y_mean']-y_grid)))[0][0]
        ax.plot(x_grid, h[:, jc]/1000, 'k', )
    else:
        ic = np.where(np.abs(pmean['x_mean']-x_grid)==np.min(np.abs(pmean['x_mean']-x_grid)))[0][0]
        ax.plot(y_grid, h[ic, :]/1000, 'k')
    
    ylim = ax.get_ylim()
    
    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean'] - y_grid) == np.min(np.abs(pmean['y_mean'] - y_grid)))[0][0]
        ax.fill_between(x_grid, h[:, jc] / 1000, 10, color='k')
    else:
        ic = np.where(np.abs(pmean['x_mean'] - x_grid) == np.min(np.abs(pmean['x_mean'] - x_grid)))[0][0]
        ax.fill_between(y_grid, h[ic, :] / 1000, 10, color='k')
        
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    if ('1967' in eddy) or ('382' in eddy):
        ax.set_xlabel('x (km')
    else:
        ax.set_xlabel('y (km)')
    
    ax.set_ylabel('Depth (km)')
    cyc = df_eddies.loc[df_eddies['ID'] == int(eddy[4:]), 'Cyc'].iloc[0]
    ax.set_title(f'{cyc}{eddy[4:]}')
    ax.invert_yaxis()
    ax.legend(title='Day')
    
    plt.tight_layout()
    plt.show()


In [ ]:
eddy = 382
d_int = 6
dic = dic_sample[f'Eddy{eddy}']
# days = np.arange(0, len(dic), d_int)
days = [10, 14, 17, 22, 30, 37, 42, 48, 56, 62]
side_profile_plotter(eddy, days)


In [ ]:
eddy = 1967
d_int = 7
dic = dic_sample[f'Eddy{eddy}']
# days = np.arange(0, len(dic), d_int)
days = [6, 14, 21, 28, 36, 42, 49, 56, 63]
side_profile_plotter(eddy, days)


In [ ]:
eddy = 926
d_int = 7
dic = dic_sample[f'Eddy{eddy}']
days = np.arange(0, len(dic), d_int)
days = [13, 16, 21, 30, 35, 40, 45, 49, 63]
side_profile_plotter(eddy, days)

In [ ]:
eddy = 896
d_int = 7
dic = dic_sample[f'Eddy{eddy}']
# days = np.arange(0, len(dic), d_int)
days = [3, 10, 14, 25, 31, 33, 36, 48, 58, 65]
side_profile_plotter(eddy, days)


In [ ]:
eddy = 2504
d_int = 8
dic = dic_sample[f'Eddy{eddy}']
days = np.arange(0, len(dic), d_int)
days = [0,  8, 16, 20, 29, 55, 59, 64, 75]
side_profile_plotter(eddy, days)


In [ ]:
eddy = 1394
d_int = 8
dic = dic_sample[f'Eddy{eddy}']
# days = np.arange(0, len(dic), d_int)
days = [0, 18, 40, 45, 48, 51, 54, 58, 64]
side_profile_plotter(eddy, days)


In [ ]:
eddy = 2749

d_int = 8
dic = dic_sample[f'Eddy{eddy}']
days = np.arange(0, len(dic), d_int)
days = [10, 24, 30, 33, 38, 48, 56, 70, 73]
side_profile_plotter(eddy, days)

In [ ]:
days

In [ ]:
eddy = 8

d_int = 23
dic = dic_sample[f'Eddy{eddy}']
# days = np.arange(0, len(dic), d_int)
days = [23, 28, 60, 100, 190]
side_profile_plotter(eddy, days)

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12,14), sharey=True)

eddy_p = sample_eddies

days_p = [[23, 28, 60, 100, 190],
          [3, 10, 14, 25, 31, 33, 36, 48, 58, 65],
          [0,  8, 16, 20, 29, 55, 59, 64, 75],
          [10, 24, 30, 33, 38, 48, 56, 70, 73],
          [10, 14, 17, 22, 30, 37, 42, 48, 56, 62],
          [13, 16, 21, 30, 35, 40, 45, 49, 63],
          [0, 18, 40, 45, 48, 51, 54, 58, 64],
          [6, 14, 21, 28, 36, 42, 49, 56, 63]]
          
for e, (eddy, days) in enumerate(zip(eddy_p, days_p)):

    ax = axs[e % 4, e // 4]

    eddy = f'Eddy{eddy}'
    
    pmean = df_means[df_means['Eddy']==eddy].iloc[0]
    
    dic = dic_sample[eddy]

    for day in dic.keys():
        if int(day[3:]) not in days:
            df = dic[day].copy()
            if ('1967' in eddy) or ('382' in eddy):
                ax.plot(df.x, -df.Depth/1000, linewidth=.5, color='k', alpha=.2)
            else:
                ax.plot(df.y, -df.Depth/1000, linewidth=.5, color='k', alpha=.2)
    
    colors = plt.get_cmap('tab10').colors
    for d, day_idx in enumerate(days):
        
        day_key = list(dic_sample[eddy].keys())[day_idx]
        df = dic[day_key].copy()
    
        if len(df):
            
            # raw data
            xs = df['x'].values
            ys = df['y'].values
            Qs = df['Q'].values
            Rcs = df['Rc'].values
            psi0s = df['psi0'].values
            depths = -df['Depth'].values / 1000  # in km
            
            # smooth against frame index t
            window = 1
            sq11  = smooth([q[0,0] for q in Qs],   depths, window=window)
            sq12  = smooth([q[0,1] for q in Qs],   depths, window=window)
            sq22  = smooth([q[1,1] for q in Qs],   depths, window=window)
            sRc   = smooth(Rcs,                    depths, window=window)
            spsi0 = smooth(psi0s,                  depths, window=window)
            
            # plot each smoothed ellipse at its depth
            if ('1967' in eddy) or ('382' in eddy):
                ax.plot(xs, depths, color=colors[d % 10], label=day_idx)
                ax.text(xs[0], -.05, day_idx, color=colors[d % 10], fontweight='bold')
            else:
                ax.plot(ys, depths, color=colors[d % 10], label=day_idx)
                ax.text(ys[0], -.05, day_idx, color=colors[d % 10], fontweight='bold')
            for x0, y0, q11, q12, q22, Rc, z0 in zip(xs, ys, sq11, sq12, sq22, sRc, depths):
                Qm = np.array([[q11, q12],[q12, q22]])
                xe, ye = plot_ellipse(Qm, (x0, y0), scale=Rc)
                ze = np.full_like(xe, z0)
                if ('1967' in eddy) or ('382' in eddy):
                    ax.plot(xe, ze, color=colors[d % 10], alpha=0.8)
                else:
                    ax.plot(ye, ze, color=colors[d % 10], alpha=0.8)

            
                
    xlim = ax.get_xlim()
    
    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean']-y_grid)==np.min(np.abs(pmean['y_mean']-y_grid)))[0][0]
        ax.plot(x_grid, h[:, jc]/1000, 'k', )
    else:
        ic = np.where(np.abs(pmean['x_mean']-x_grid)==np.min(np.abs(pmean['x_mean']-x_grid)))[0][0]
        ax.plot(y_grid, h[ic, :]/1000, 'k')
    
    ylim = ax.get_ylim()
    
    if ('1967' in eddy) or ('382' in eddy):
        jc = np.where(np.abs(pmean['y_mean'] - y_grid) == np.min(np.abs(pmean['y_mean'] - y_grid)))[0][0]
        ax.fill_between(x_grid, h[:, jc] / 1000, 10, color='k')
    else:
        ic = np.where(np.abs(pmean['x_mean'] - x_grid) == np.min(np.abs(pmean['x_mean'] - x_grid)))[0][0]
        ax.fill_between(y_grid, h[ic, :] / 1000, 10, color='k')
        
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    if ('1967' in eddy) or ('382' in eddy):
        ax.set_xlabel('x (km)')
    else:
        ax.set_xlabel('y (km)')
    
    ax.set_ylabel('Depth (km)')
    cyc = df_eddies.loc[df_eddies['ID'] == int(eddy[4:]), 'Cyc'].iloc[0]
    ax.set_title(f'{cyc}{eddy[4:]}')
    # ax.legend(title='Day', loc='lower right', ncol=2)
    
axs[0,0].invert_yaxis()
axs[0,0].set_ylim(None, -.3)
    
plt.tight_layout()
plt.show()
    